In [69]:
# 式（5）
def get_Q_out_H_hs(_equipment, _Q_dmd_H_hs):
    
    if _equipment == 'gasConv' or 'gasCondensing':
        
    
    
    
    min(_Q_dmd_H_hs , _Q_max_H_hs)

In [70]:
get_Q_out_H_hs(_Q_dmd_H_hs = 10,_Q_max_H_hs = 2)

2

In [47]:
# 式（7）　温水供給運転率
def get_r_WS_hs(_Q_dmd_H_hs, _r_WS_ln_i):
    if _Q_dmd_H_hs > 0:
        
        # 放熱系統の数を温水供給運転率の値の数で判定
        rad_num = 0
        for i in _r_WS_ln_i:
            if type(i) == int or type(i) == float:
                rad_num += 1
                _r_WS_hs = i

        if rad_num == 1:
            return _r_WS_hs
        elif rad_num >= 2:
            return 1
    elif _Q_dmd_H_hs == 0:
        return 0

#### Example

In [56]:
get_r_WS_hs(_Q_dmd_H_hs=1, _r_WS_ln_i=[None,None, 0.2])

0.2